# 🔍 Recuperación de Documentos - Catálogo de Productos de Motos

En este notebook vamos a explorar y demostrar las capacidades de recuperación de documentos usando **vectores híbridos** (densos + dispersos) combinados con **filtros avanzados** sobre nuestro catálogo de productos de motos.

#### 🎯 Objetivos

1. **Explorar la colección indexada** - Conocer los valores únicos para filtros
2. **Búsquedas básicas** - Consultas simples por texto
3. **Filtros individuales** - Marca, categoría, precio, etc.
4. **Filtros combinados** - Múltiples criterios simultáneos
5. **Búsquedas híbridas avanzadas** - Vectores + filtros complejos

#### 📊 Colección: `repuesto_motos_mundibot`
- **Documentos indexados**: ~10,000 productos
- **Vectores densos**: OpenAI text-embedding-3-small (1536D)
- **Vectores dispersos**: SPLADE (Sparse Learned And Interpretable)
- **Filtros disponibles**: Marca, modelo, categoría, precio, dimensiones, etc.


## 1. Configuración inicial

In [ ]:
import pandas as pd
import logging
from typing import List, Dict, Any
from src.core.domains.products import ProductSearchService, PRODUCTS_SCHEMA
from src.core.retriever import create_collection_config, HybridRetriever
from src.core.retriever.collection_manager import CollectionManager
from src.core.settings import settings
from qdrant_client import models

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# --- Función para mostrar resultados de manera clara ---
def mostrar_resultados(response, titulo: str):
    """
    Mostrar resultados de búsqueda de forma organizada.
    
    Args:
        response: SearchResponse del retriever
        titulo: Título descriptivo para la búsqueda
    """
    print(f"📋 {titulo}")
    print(f"   Resultados encontrados: {len(response.results)}")
    
    if hasattr(response, 'search_time_ms') and response.search_time_ms:
        print(f"   Tiempo de búsqueda: {response.search_time_ms:.1f}ms")
    
    print()
    
    if not response.results:
        print("   ❌ No se encontraron resultados")
        return
    
    for i, result in enumerate(response.results, 1):
        payload = result.payload
        titulo_producto = payload.get('titulo', 'Sin título')
        marca = payload.get('marca', 'Sin marca')
        precio = payload.get('precio', 0)
        categoria = payload.get('categoria', 'Sin categoría')
        
        print(f"   {i}. {titulo_producto}")
        print(f"      Marca: {marca} | Categoría: {categoria}")
        print(f"      Precio: ${precio:,.0f} COP | Score: {result.score:.3f}")
        
        # Mostrar marcas compatibles si existen
        marcas_lista = payload.get('marcas_lista', [])
        if marcas_lista:
            marcas_str = ', '.join(marcas_lista[:3])  # Mostrar solo las primeras 3
            if len(marcas_lista) > 3:
                marcas_str += f" (+{len(marcas_lista)-3} más)"
            print(f"      Compatible con: {marcas_str}")
        
        print()

In [ ]:
def mostrar_resultados_filtros(response, titulo: str, filtros_aplicados: dict = None):
    """
    Mostrar resultados con información sobre filtros aplicados.
    
    Args:
        response: SearchResponse del retriever
        titulo: Título descriptivo para la búsqueda
        filtros_aplicados: Diccionario con los filtros que se aplicaron
    """
    print(f"📋 {titulo}")
    print(f"   Resultados encontrados: {len(response.results)}")
    
    if filtros_aplicados:
        print("   🔍 Filtros aplicados:")
        for filtro, valor in filtros_aplicados.items():
            if valor is not None:
                print(f"      • {filtro}: {valor}")
    
    if hasattr(response, 'search_time_ms') and response.search_time_ms:
        print(f"   ⏱️ Tiempo de búsqueda: {response.search_time_ms:.1f}ms")
    
    print()
    
    if not response.results:
        print("   ❌ No se encontraron resultados con estos filtros")
        return
    
    for i, result in enumerate(response.results, 1):
        payload = result.payload
        titulo_producto = payload.get('titulo', 'Sin título')
        marca = payload.get('marca', 'Sin marca')
        precio = payload.get('precio', 0)
        categoria = payload.get('categoria', 'Sin categoría')
        tipo_repuesto = payload.get('tipo_repuesto', 'N/A')
        
        print(f"   {i}. {titulo_producto}")
        print(f"      Marca: {marca} | Categoría: {categoria} | Tipo: {tipo_repuesto}")
        print(f"      Precio: ${precio:,.0f} COP | Score: {result.score:.3f}")
        
        # Mostrar marcas compatibles
        marcas_lista = payload.get('marcas_lista', [])
        if marcas_lista:
            marcas_str = ', '.join(marcas_lista[:3])
            if len(marcas_lista) > 3:
                marcas_str += f" (+{len(marcas_lista)-3} más)"
            print(f"      Compatible con: {marcas_str}")
        
        print()

#### **OPCIÓN 1**: Usando ProductSearchService

In [ ]:
print("🔧 Configuración con ProductSearchService:")

# Inicialización simple - solo necesitas el nombre de la colección
product_service = ProductSearchService("repuesto_motos_mundibot")

print(f"   Colección: {product_service.collection_name}")
print(f"   Vector denso: {product_service.config.dense_vector_name}")
print(f"   Vector disperso: {product_service.config.sparse_vector_name}")
print(f"   Modelo OpenAI: {product_service.config.openai_model}")

# Verificar estado de la colección
collection_manager = CollectionManager(product_service.config)
if collection_manager.collection_exists():
    total_docs = collection_manager.count_points(exact=True)
    print(f"   Documentos en colección: {total_docs:,}")
    print("✅ Colección lista para búsquedas con ProductSearchService!")
else:
    print("❌ Error: La colección no existe. Ejecuta primero el notebook de indexación.")
    raise ValueError("Collection not found")

#### **OPCIÓN 2**: Usando HybridRetriever genérico

In [ ]:
# --- OPCIÓN 2: Usando HybridRetriever genérico ---

# Configuración genérica usando la nueva función
generic_config = create_collection_config(
    collection_name="repuesto_motos_mundibot",
    payload_indices=PRODUCTS_SCHEMA,
    # qdrant_url se toma automáticamente de settings
    # Todos los demás parámetros tienen defaults inteligentes
)

# Inicializar retriever genérico
retriever = HybridRetriever(generic_config)

print(f"   Colección: {generic_config.collection_name}")
print(f"   Qdrant URL: {generic_config.qdrant_url}")
print(f"   Configuración automática desde settings: ✅")

# Verificar estado
collection_manager_generic = CollectionManager(generic_config)
if collection_manager_generic.collection_exists():
    total_docs = collection_manager_generic.count_points(exact=True)
    print(f"   Documentos en colección: {total_docs:,}")
    print("✅ Colección lista para búsquedas genéricas!")

# --- Mostrar configuración centralizada ---
print("\n📋 Configuración centralizada desde settings:")
print(f"   Qdrant: {settings.qdrant_host}:{settings.qdrant_port}")
print(f"   Modelo embedding: {settings.embedding_model}")
print(f"   Dimensiones: {settings.embedding_dimensions}")
print(f"   Modelo sparse: {settings.sparse_model}")
print(f"   Batch size: {settings.default_batch_size}")
print(f"   API key configurada: {'✅' if settings.openai_api_key else '❌'}")

print("\n🎉 Sistema listo! Usa:")
print("   • product_service.search_products() para búsquedas específicas de productos")
print("   • retriever.search() para búsquedas genéricas avanzadas")

##  2. Exploracion de la Coleccion

Antes de realizar búsquedas, vamos a explorar qué valores únicos tenemos disponibles para cada campo de filtro. Esto nos ayudará a entender el catálogo y crear filtros efectivos

In [ ]:
# --- Exploración de Valores Únicos con Nueva Arquitectura ---
print("Explorando valores únicos en la colección...\n")

def get_unique_values(field_name: str, limit: int = 50) -> List[Any]:
    """
    Obtener valores únicos de un campo usando la nueva arquitectura.
    
    Args:
        field_name: Nombre del campo a explorar
        limit: Máximo número de valores únicos a retornar
    """
    try:
        # Usar el collection_manager ya inicializado
        results = collection_manager.client.scroll(
            collection_name=product_service.collection_name,
            limit=1000,  # Muestra representativa
            with_payload=True,
            with_vectors=False
        )
        
        # Extraer valores únicos del campo
        values = set()
        for point in results[0]:  # results es (points, next_page_offset)
            if hasattr(point, 'payload') and point.payload:
                value = point.payload.get(field_name)
                if value is not None:
                    # Si es una lista, agregar cada elemento
                    if isinstance(value, list):
                        values.update(value)
                    else:
                        values.add(value)
        
        # Convertir a lista ordenada y limitar
        unique_values = sorted(list(values))[:limit]
        return unique_values
        
    except Exception as e:
        logger.error(f"Error obteniendo valores para {field_name}: {e}")
        return []

# Campos principales para explorar (nombres exactos de la colección)
campos_explorar = {
    'marcas_lista': 'Marcas de motos compatibles',
    'marca_original': 'Marcas originales de productos', 
    'categoria': 'Categorías de repuestos',
    'subcategoria': 'Subcategorías específicas',
    'tipo_repuesto': 'Tipos de repuesto (ORIGINAL/GENERICO)',
    'modelos_lista': 'Modelos de motos compatibles',
    'es_llanta': 'Indicador de llantas (True/False)'
}

print("📋 Valores únicos por campo:\n")
valores_unicos = {}

for campo, descripcion in campos_explorar.items():
    print(f"• {descripcion} (campo: {campo}):")
    valores = get_unique_values(campo)
    valores_unicos[campo] = valores
    
    if valores:
        print(f"  Total únicos: {len(valores)}")
        # Mostrar primeros 10 valores
        for i, valor in enumerate(valores[:10]):
            print(f"  {i+1:2d}. {valor}")
        if len(valores) > 10:
            print(f"  ... y {len(valores)-10} valores más")
    else:
        print("  ❌ No se encontraron valores")
    print()

In [ ]:
valores = get_unique_values("categoria")
print(valores)

In [ ]:
# --- Exploración de Rangos de Precios ---
print("💰 Análisis de precios:\n")

def get_price_stats() -> Dict[str, float]:
    """Obtener estadísticas completas de precios."""
    try:
        results = collection_manager.client.scroll(
            collection_name=product_service.collection_name,
            limit=2000,  # Muestra grande para estadísticas confiables
            with_payload=True,
            with_vectors=False
        )
        
        # Extraer precios válidos
        precios = []
        for point in results[0]:
            if hasattr(point, 'payload') and point.payload:
                precio = point.payload.get('precio')
                if precio is not None and isinstance(precio, (int, float)) and precio > 0:
                    precios.append(precio)
        
        if precios:
            precios.sort()
            n = len(precios)
            return {
                'min': min(precios),
                'max': max(precios),
                'promedio': sum(precios) / n,
                'mediana': precios[n//2],
                'q1': precios[n//4],
                'q3': precios[3*n//4],
                'count': n
            }
        return {}
            
    except Exception as e:
        logger.error(f"Error obteniendo estadísticas de precios: {e}")
        return {}

# Obtener y mostrar estadísticas de precios
stats_precios = get_price_stats()

if stats_precios:
    print("📊 Estadísticas de Precios (COP):")
    print(f"   Precio mínimo: ${stats_precios['min']:,.0f}")
    print(f"   Precio máximo: ${stats_precios['max']:,.0f}")
    print(f"   Precio promedio: ${stats_precios['promedio']:,.0f}")
    print(f"   Precio mediano: ${stats_precios['mediana']:,.0f}")
    print(f"   Q1 (25%): ${stats_precios['q1']:,.0f}")
    print(f"   Q3 (75%): ${stats_precios['q3']:,.0f}")
    print(f"   Productos con precio: {stats_precios['count']:,}")
    
    print(f"\n💡 Rangos sugeridos para filtros:")
    print(f"   Económico: $0 - ${stats_precios['q1']:,.0f}")
    print(f"   Medio: ${stats_precios['q1']:,.0f} - ${stats_precios['q3']:,.0f}")
    print(f"   Premium: ${stats_precios['q3']:,.0f} - ${stats_precios['max']:,.0f}")
    
    # Guardar stats para uso posterior
    globals()['price_ranges'] = {
        'economico_max': stats_precios['q1'],
        'medio_min': stats_precios['q1'],
        'medio_max': stats_precios['q3'],
        'premium_min': stats_precios['q3']
    }
else:
    print("❌ No se pudieron obtener estadísticas de precios")

print(f"\n✅ Exploración completada!")
print(f"📝 Usa estos valores para crear filtros efectivos en las búsquedas")

## 2. Búsquedas Básicas con Vectores Híbridos

Ahora que conocemos la estructura de datos, vamos a realizar búsquedas usando vectores híbridos (densos + dispersos) sin filtros para ver la capacidad semántica del sistema.

In [ ]:
# --- Búsquedas Básicas con ProductSearchService ---
print("Realizando búsquedas básicas con ProductSearchService:\n")

# Ejemplo 1: Búsqueda de comandos/controles
query1 = "comando derecho moto pulsar"
print(f"Ejecutando: await product_service.search_products('{query1}', limit=5)")
response1 = await product_service.search_products(query1, limit=5)
mostrar_resultados(response1, f"Búsqueda: '{query1}'")

### Comparación de Tipos de Búsqueda

In [ ]:
# --- Comparación de Tipos de Búsqueda ---
print("\n🔬 Comparando tipos de búsqueda para el mismo query:\n")

query_test = "pastillas de freno yamaha"

# Búsqueda híbrida (recomendada)
print(f"Ejecutando híbrida: await product_service.search_products('{query_test}', search_type='hybrid')")
response_hybrid = await product_service.search_products(query_test, search_type="hybrid", limit=3)
mostrar_resultados(response_hybrid, "Búsqueda Híbrida (Dense + Sparse)")

# Búsqueda solo densa
print(f"Ejecutando densa: await product_service.search_products('{query_test}', search_type='dense')")
response_dense = await product_service.search_products(query_test, search_type="dense", limit=3)
mostrar_resultados(response_dense, "Búsqueda Solo Densa (Semántica)")

# Búsqueda solo sparse
print(f"Ejecutando sparse: await product_service.search_products('{query_test}', search_type='sparse')")
response_sparse = await product_service.search_products(query_test, search_type="sparse", limit=3)
mostrar_resultados(response_sparse, "Búsqueda Solo Sparse (Palabras clave)")

##  3. Exploración de Búsquedas con Filtros

En esta sección exploraremos cómo combinar filtros específicos con diferentes tipos de búsqueda vectorial. Esto nos permite obtener resultados más precisos y relevantes para casos de uso específicos.

#### Tipos de Búsqueda con Filtros:
- **Solo Filtros** (`filter_only`): Búsqueda basada únicamente en filtros, sin vectores
- **Filtros + Dense** (`dense`): Búsqueda semántica + filtros específicos  
- **Filtros + Sparse** (`sparse`): Búsqueda por palabras clave + filtros específicos
- **Filtros + Hybrid** (`hybrid`): Búsqueda semántica + palabras clave + filtros (recomendado)

In [ ]:
# --- 1. BÚSQUEDA SOLO CON FILTROS (SIN VECTORES) ---
print("🔍 1. BÚSQUEDA SOLO CON FILTROS (filter_only):")
print("   Busca productos usando únicamente filtros, sin análisis de texto\n")

# Ejemplo: Solo repuestos YAMAHA de categoría ELECTRICO
filtros_yamaha_electrico = {
    'marcas_lista': ['YAMAHA'], 
    'tipo_repuesto': 'ORIGINAL'
}

response_filter_only = await product_service.search_products(
    query="",  # Query vacío para filter_only
    search_type="filter_only",
    marcas_lista=filtros_yamaha_electrico['marcas_lista'],
    tipo_repuesto=filtros_yamaha_electrico['tipo_repuesto'],
    limit=5
)

mostrar_resultados_filtros(
    response_filter_only, 
    "Solo Filtros: Repuestos YAMAHA Eléctricos Originales",
    filtros_yamaha_electrico
)

In [ ]:
# --- 2. BÚSQUEDA DENSE + FILTROS ---
print("🔍 2. BÚSQUEDA SEMÁNTICA + FILTROS (dense):")
print("   Combina búsqueda semántica con filtros específicos\n")

query_bateria = "batería recargable 12 voltios"
filtros_bateria = {
    'marcas_lista': ['YAMAHA', 'HONDA', 'SUZUKI'],
    'precio_max': 40000.0
}

response_dense_filter = await product_service.search_products(
    query=query_bateria,
    search_type="dense",
    marcas_lista=filtros_bateria['marcas_lista'],
    precio_max=filtros_bateria['precio_max'],
    limit=5
)

mostrar_resultados_filtros(
    response_dense_filter,
    f"Dense + Filtros: '{query_bateria}' con filtros",
    filtros_bateria
)

In [ ]:
# --- 3. BÚSQUEDA SPARSE + FILTROS ---
print("🔍 3. BÚSQUEDA PALABRAS CLAVE + FILTROS (sparse):")
print("   Combina búsqueda por palabras clave exactas con filtros\n")

query_pastillas = "batería recargable 12 voltios"
filtros_frenos = {
    'marcas_lista': ['YAMAHA'],
    'categoria': ['ELECTRICO / ENCENDIDO'],
}

response_sparse_filter = await product_service.search_products(
    query=query_pastillas,
    search_type="sparse", 
    marcas_lista=filtros_frenos['marcas_lista'],
    categoria=filtros_frenos['categoria'],
    limit=5
)

mostrar_resultados_filtros(
    response_sparse_filter,
    f"Sparse + Filtros: '{query_pastillas}' con filtros",
    filtros_frenos
)

In [ ]:
# --- 4. BÚSQUEDA HÍBRIDA + FILTROS (RECOMENDADA) ---
print("🔍 4. BÚSQUEDA HÍBRIDA + FILTROS (hybrid) - RECOMENDADA:")
print("   Combina semántica + palabras clave + filtros para máxima precisión\n")

query_aceite = "aceite motor sintético"
filtros_aceite = {
    'categoria': ['MOTOR INTERNO'],
    'tipo_repuesto': 'ORIGINAL',
    'precio_min': 30000.0,
    'precio_max': 150000.0
}

response_hybrid_filter = await product_service.search_products(
    query=query_aceite,
    search_type="hybrid",
    # categoria=filtros_aceite['categoria'],
    tipo_repuesto=filtros_aceite['tipo_repuesto'],
    precio_min=filtros_aceite['precio_min'],
    precio_max=filtros_aceite['precio_max'],
    limit=5
)

mostrar_resultados_filtros(
    response_hybrid_filter,
    f"Hybrid + Filtros: '{query_aceite}' con filtros",
    filtros_aceite
)
